In [1]:
import dash
import dash_bootstrap_components as dbc
import numpy as np
import pandas as pd
import plotly.express as px
from dash import Input, Output, dcc, html
from sklearn.linear_model import QuantileRegressor

import plotly.graph_objects as go


In [2]:
data_source_color = {
    "재무상태": "Olive",
    "손익계산": "DarkGreen",
    "현금흐름": "DarkSlateBlue",
    "주식": "Gray",
    "거래소": "Brown",
    "PerShare": "Aqua",
    "Multiple": "CadetBlue",
    "안정성": "Crimson",
    "수익성": "DeepPink",
}
data_source = data_source_color.keys()

data_years = ["2018", "2019", "2020", "2021", "2022"]

In [3]:

# load function formula data
function_data = pd.read_excel("../data/기업정보 데이터 함수식.xlsx")
function_data["분류"] = function_data["분류"].fillna(method="ffill")

In [4]:

# make default data
xls = pd.ExcelFile("../data/기업데이터수집_2차전지업체_230526.xlsx")

In [5]:
def cal_ratio(
    df,
    name: str,
    numerator: str,
    denominator: str = np.nan,  # If np.nan, use the previous numerator value
    to_percentage: bool = False,
    data_years: list = ["2018", "2019", "2020", "2021", "2022"],
) -> pd.DataFrame:
    if denominator is np.nan:
        result = df.loc[df.항목명 == numerator, data_years]
        denominator = result.copy()
        denominator.columns = [str(int(cnt) + 1) for cnt in data_years]
        result = result / denominator.replace(0, np.nan)  # prevent ZeroDivisionError
        result = result[data_years]
    else:
        result = pd.DataFrame(
            df.loc[df.항목명 == numerator, data_years].values
            / df.loc[df.항목명 == denominator, data_years]
            .replace(0, np.nan)
            .values,  # prevent ZeroDivisionError,
            columns=data_years,
        )

    if to_percentage:
        result *= 100

    if result.empty:
        result = pd.DataFrame({column: [np.nan] for column in result.columns})

    result["항목명"] = name

    return result

In [6]:

data = {}
for cnt in xls.sheet_names:
    df = pd.read_excel(xls, cnt)
    df.name = cnt
    df = df.loc[df.분류.isin(data_source), ["항목명", *data_years]]
    df_n_stocks = df.loc[df.항목명.isin(["발행주식수(보통주)", "발행주식수(우선주)"]), data_years].sum()
    df_n_stocks["항목명"] = "발행주식수"

    df = pd.concat([df, pd.DataFrame(df_n_stocks).T], ignore_index=True)

    # concat ratio values
    for _, row in function_data.iterrows():
        df = pd.concat(
            [
                df,
                cal_ratio(
                    df,
                    name=row["name"],
                    numerator=row["numerator"],
                    denominator=row["denominator"],
                    to_percentage=row["to_percentage"],
                ),
            ],
            ignore_index=True,
        )
    data[cnt] = df



In [7]:
data

{'엘지에너지솔루션(연결)':           항목명 2018 2019      2020        2021        2022
 0        자산총계    0    0  19941795    23764137    38299445
 1        유동자산    0    0   9085706     9535827    18804269
 2    현금및현금성자산    0    0   1493145     1282880     5937967
 3        매출채권    0    0   2931150     2914458     4771846
 4      단기금융상품    0    0    600000        1067           0
 ..        ...  ...  ...       ...         ...         ...
 71     매출액증가율  NaN  NaN       NaN         NaN         NaN
 72    영업이익증가율  NaN  NaN       NaN -161.713026  157.939672
 73  EBITDA증가율  NaN  NaN       NaN  -471.65929  123.080255
 74     EPS증가율  NaN  NaN       NaN         NaN         NaN
 75     BPS증가율  NaN  NaN       NaN         NaN         NaN
 
 [76 rows x 6 columns],
 '삼성SDI(연결)':           항목명         2018         2019         2020        2021         2022
 0        자산총계  19349721.18  19852096.45  21534232.27    25833193  30257525.17
 1        유동자산   5519342.21    5181414.9   5657405.17  7444907.07   9651702.38


In [11]:
data['엘지에너지솔루션(연결)']

,항목명,2018,2019,2020,2021,2022
0,자산총계,0,0,19941795,23764137,38299445
1,유동자산,0,0,9085706,9535827,18804269
2,현금및현금성자산,0,0,1493145,1282880,5937967
3,매출채권,0,0,2931150,2914458,4771846
4,단기금융상품,0,0,600000,1067,0
...,...,...,...,...,...,...
71,매출액증가율,NaN,NaN,NaN,NaN,NaN
72,영업이익증가율,NaN,NaN,NaN,-161.713026,157.939672
73,EBITDA증가율,NaN,NaN,NaN,-471.65929,123.080255
74,EPS증가율,NaN,NaN,NaN,NaN,NaN


In [3]:
domain_map = pd.read_csv('../data/domain_map.csv')

In [4]:
nested_dict = {}
for _, row in domain_map.iterrows():
    category = row['category']
    division = row['division']
    sub = row['sub']
    name = row['name']
    percent = row['percent']

    category_dict = nested_dict.setdefault(category, [])
    division_dict = next((item for item in category_dict if item['division'] == division and item['sub'] == sub), None)
    if not division_dict:
        division_dict = {'division': division, 'sub': sub, 'contents': []}
        category_dict.append(division_dict)

    division_dict['contents'].append({'name': name, 'percent': percent})

nested_dict

{'자원생산': [{'division': '리튬',
   'sub': '희귀',
   'contents': [{'name': '아르헨티나', 'percent': 37.0},
    {'name': '호주', 'percent': 23.0},
    {'name': '칠레', 'percent': 22.0},
    {'name': '중국', 'percent': 11.0},
    {'name': '볼리비아', 'percent': 7.0}]},
  {'division': '니켈',
   'sub': '지역편중',
   'contents': [{'name': '인도네시아', 'percent': 21.0},
    {'name': '필리핀', 'percent': 19.0},
    {'name': '러시아', 'percent': 11.0},
    {'name': '캐나다', 'percent': 9.0},
    {'name': '가나', 'percent': 9.0}]},
  {'division': '코발트',
   'sub': '희귀',
   'contents': [{'name': '콩고', 'percent': 60.0},
    {'name': '러시아', 'percent': 10.0},
    {'name': '인도네시아', 'percent': 9.0},
    {'name': '몽골', 'percent': 7.0},
    {'name': '캐나다', 'percent': 6.0}]},
  {'division': '망간',
   'sub': '희소',
   'contents': [{'name': '중국', 'percent': 53.0},
    {'name': '남아프리카', 'percent': 22.0},
    {'name': '가나', 'percent': 10.0},
    {'name': '호주', 'percent': 8.0},
    {'name': '인도', 'percent': 4.0}]},
  {'division': '흑연',
   'sub': nan

In [5]:
for n in nested_dict:
    print(n)

자원생산
원자재 가공
중간재 생산
설비/장비 공급
최종재 생산
타겟 세그먼트
폐기 및 재활용
